In [104]:
import numpy as np
import scipy.misc
import glob
import sys
import tensorflow as tf
from datetime import datetime
from pathlib import Path
import os
from os.path import basename
import collections

In [105]:
def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def check_files(folder):
    """
    Given path to folder, returns whether it's empty or not
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    if(len(filenames)==0):
        return False
    return True

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(label2id, label):
    """
    Returns label for a folder
    """
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [109]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_image_map(folder, label):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    image_map = {}
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f) / 255.0
        #img_arr = img_arr.flatten() / 255.0
        name = basename(f)
        time = os.path.getctime(f)
        image_map[time] = [img_arr, label]
        #images.append(img_arr)
    #X = np.column_stack(images)

    return image_map

def get_train_data(data_root_path, label_mapping_path):
    """
    Return X and y
    """
    X = []
    y = []
    labels = os.listdir(data_root_path)
    id2label, label2id = get_label_mapping(label_mapping_path)
    print(label2id)
    image_map = {}
    for label in labels:
        train_data_path = data_root_path + label
        if(check_files(train_data_path)) :
            temp_map = get_image_map(train_data_path, label)
            image_map.update(temp_map)
    new_map = collections.OrderedDict(sorted(image_map.items()))
    
    for k, v in new_map.items():
        print(k)
        X.append(v[0])
        y.append(get_label(label2id, v[1]))
    X = np.array(X)
    return X, np.array(y)

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)# Load the data

In [119]:
# Load the data
# The data folder can have any number of subfolders. 
#The name of the subfolder will be treated as the label for all the images in that subfolder.
#Add all the subfolder names in labels.txt
data_root_path = './testdata/' #will take all the folders in this directory to be used as labels
label_mapping_path = './labels.txt' #labels.txt should NOT be in the data_root_path
X_train, y_train = get_train_data(data_root_path, label_mapping_path) # this may take a few minutes
print(X_train.shape)
print(X_train)
print(y_train)
print('Data loading done')

{'s': 3, 'w': 1, 'd': 2, 'a': 0}
1511994131.5324707
1511994135.5425086
(2, 480, 720, 3)
[[[[ 0.24313725  0.38039216  0.2627451 ]
   [ 0.25098039  0.38039216  0.2745098 ]
   [ 0.23921569  0.35686275  0.2627451 ]
   ..., 
   [ 0.5254902   0.66666667  0.5254902 ]
   [ 0.51372549  0.65882353  0.5254902 ]
   [ 0.50196078  0.65882353  0.5254902 ]]

  [[ 0.24705882  0.34117647  0.20784314]
   [ 0.23137255  0.3372549   0.21568627]
   [ 0.20784314  0.3254902   0.22352941]
   ..., 
   [ 0.31764706  0.43529412  0.29411765]
   [ 0.31372549  0.43137255  0.29803922]
   [ 0.31764706  0.43529412  0.30196078]]

  [[ 0.21176471  0.26666667  0.1254902 ]
   [ 0.2         0.28235294  0.15294118]
   [ 0.17254902  0.28627451  0.17647059]
   ..., 
   [ 0.30196078  0.39607843  0.25490196]
   [ 0.31372549  0.4         0.25882353]
   [ 0.31372549  0.39607843  0.26666667]]

  ..., 
  [[ 0.94117647  0.94901961  0.9372549 ]
   [ 0.94117647  0.94901961  0.9372549 ]
   [ 0.94117647  0.94901961  0.9372549 ]
   ..., 
 

/root/anaconda3/envs/python3/lib/python3.5/site-packages/ipykernel/__main__.py:5: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
